In [5]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC, OneClassSVM
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
import os
import random as python_random
import argparse
import numpy as np
import os, sys, time
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

In [6]:
import os
import pickle
import glob
import pandas as pd
import xml.etree.ElementTree as ET

# Construct empty DataFrame
df_sentences = pd.DataFrame(columns=[
    'document', 'file_path', 'split', 'text', 'type'
])

def parse_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    documents = []

    for elem in root.iter():
        if elem.tag == "OP" or elem.tag == "reply":
            all_descendants = [b for b in elem.iter() if b is not elem]
            for desc in all_descendants:
                documents.append((desc.text, desc.tag))
    return documents

# Retrieve premises from all XML files
for split in ['positive', 'negative']:
    for path in glob.glob(os.path.join(f'v2.0/{split}', '*.xml')):
        try:
            for premise, argument in parse_xml(path):
                df_sentences = pd.concat([df_sentences, pd.DataFrame({
                    'document': [path],
                    'file_path': [path],
                    'split': [path.split("/")[1]],
                    'text': [premise],
                    'type': [argument],
                })], axis=0, ignore_index=True)
        except ET.ParseError:
            pass

pickle.dump( df_sentences, open( "./v2.0-processed/df_sentences.pickle", "wb" ) )
#df_sentences.to_pickle('./v2.0-processed/df_sentences.pickle')

In [7]:
df = df_sentences

# Preprocess the train data
documents = df["text"].tolist()
labels = df["type"].tolist()


X_train, X_test, y_train, y_test = train_test_split(documents, labels, test_size=0.2, random_state=1, stratify=labels)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1, stratify=y_train) # 0.25 x 0.8 = 0.2

random_state = 44

# Construct pipeline
pipe = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svc', LinearSVC(random_state=random_state)),
])

# Convert labels to numerical values
#encoder = LabelBinarizer()
#y_train_bin = encoder.fit_transform(y_train)
#y_train = np.argmax(y_train_bin, axis=1)

pipe.fit(X_train, y_train)

# Predict dev
y_pred = pipe.predict(X_val)
print(classification_report(y_val, y_pred))

# Predict test
#y_pred = pipe.predict(X_test)
#print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       claim       0.69      0.63      0.66       317
     premise       0.73      0.78      0.75       413

    accuracy                           0.71       730
   macro avg       0.71      0.70      0.71       730
weighted avg       0.71      0.71      0.71       730



In [88]:
df_sentences

,document,file_path,split,text,type
0,v2.0/positive/88.xml,v2.0/positive/88.xml,positive,[47% of all jobs are at risk of being automate...,premise
1,v2.0/positive/88.xml,v2.0/positive/88.xml,positive,This number will grow grow until the vast majo...,premise
2,v2.0/positive/88.xml,v2.0/positive/88.xml,positive,Since some of this automation will inevitably ...,premise
3,v2.0/positive/88.xml,v2.0/positive/88.xml,positive,By fortifying themselves in their gated commun...,premise
4,v2.0/positive/88.xml,v2.0/positive/88.xml,positive,"Once everyone is dead, they can simply be wipe...",premise
...,...,...,...,...,...
3642,v2.0/negative/85.xml,v2.0/negative/85.xml,negative,If the urban folk get rid of the rural folk th...,claim
3643,v2.0/negative/85.xml,v2.0/negative/85.xml,negative,They couldn't just survive together,claim
3644,v2.0/negative/85.xml,v2.0/negative/85.xml,negative,the ecosystem will not be able to sustain the ...,premise
3645,v2.0/negative/85.xml,v2.0/negative/85.xml,negative,The rural folk can survive without the urban folk,claim
